# 这个文件处理：
这段数据的模拟时间是600$\Omega_{cp}^{-1}$ ,将数据截断至450$\Omega_{cp}^{-1}$

scalar数据记录：
1. 读取scalar.txt的量，并记录电磁场，U_tot， Ukin_ions的能量密度 （ E（t) ）；

2. 读取particle binning的数据，记录三种粒子的垂直，平行温度以及A（温度各向异性）

field_0数据记录：
1. 读取field0，数据空间间隔，x轴y轴分别间隔原来两倍，数据量会缩小原来的4倍，后续用作分析波形特点，频率和空间波数

field_1数据记录：
1. 读取field0，数据重采样的时间间隔成原来的10倍，数据量会缩小成原来的10倍，后续用作空间滤波

In [ ]:
# numerics
import numpy as np
import math as m
import scipy
from scipy.fftpack import fft,ifft,fftshift,fft2
from scipy.fft import fft, fftfreq
from scipy.signal import butter, filtfilt
from scipy import signal
pi = m.pi

# plot
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator, AutoMinorLocator
from matplotlib.gridspec import GridSpec
from matplotlib.patches import FancyArrowPatch, ArrowStyle
import matplotlib.patches as patches
from wav_analysis.add_cb import add_color_bar_V
from cmap import purula

# others
import os
import happi
import h5py

plt.style.use("paper")

In [2]:
path = os.getcwd()

# Open a Scalar diagnostic
S = happi.Open(path)
filename = '20ns_scalar'

Loaded simulation '/media/ustcxp/C14D581BDA18EBFA/research/MS_wave/para_MS/ns/20ns'
Scanning for Scalar diagnostics
Scanning for Field diagnostics
Scanning for Probe diagnostics
Scanning for ParticleBinning diagnostics
Scanning for RadiationSpectrum diagnostics
Scanning for Performance diagnostics
Scanning for Screen diagnostics
Scanning for Tracked particle diagnostics


In [ ]:
# basic parameters
me = 1.
e = 1.
mp = 100 * me
c = 1
# -> wpe = 1

wpp = 1./np.sqrt(mp)  # proton plasma frequency
v_ratio = 15.         # c/vA
wcp = wpp/v_ratio     # cyclotron frequency
B0 = wcp*mp/e         # magnetic field magnitude

vA = c/v_ratio        # proton Afven speed
vbth = 0.03*vA        # background proton thermal velocity
vsth = 0.45*vA        # shell dist. proton thermal velocity
dt = 0.001            # simualtion time step, in wcp^-1

#使用初始背景磁场的大小来初步验证模拟结果
init_Bx = B0                     # B0 is on x axis
lambda_p = c/wpp             # characteristc length (proton inertial length)

# simulation length & grid 
Lx = 102.4*lambda_p         
Ly = 12.8*lambda_p
Nx = 512
Ny = 128
dx = Lx/Nx
dy = Ly/Ny

#setting grid
#空间上的单位是lambda_i
dx_norm = dx/lambda_p
dy_norm = dy/lambda_p
x = np.linspace(0,Nx,Nx+1)
y = np.linspace(0,Ny,Ny+1)

X,Y = np.meshgrid(x.astype(int),y.astype(int))


# 读取scalar.txt中的Bx, 计算初始磁场总能量来进行验算
UBx = S.Scalar('Uelm_Bx_m').getData()
Utot = S.Scalar('Utot').getData()
init_U = Utot[0]

# integrate 
init_UB = 0.5 * init_Bx**2 * dx * dy * (Nx+1) * (Ny+2)

print('Uelm_Bx_m[t=0] = ', {UBx[0]})
print('integrate inital UB = ', {init_UB})

In [ ]:
# 读取scalar.txt中的By, Bz, Ex, Ey, Ez, Ukin_shell_ion, Ukin_cold_ion

wcp_unit = '$\Omega_{cp}$'
# every_field0 = 50
Bx = np.array(S.Field(0,'Bx').getData())
field0_t = np.array(S.Field(0,'Bx').getTimes())

# unit transfer to \Omega_cp
# field0_t * wcp

# every_field1 = 500
UBy = S.Scalar('Uelm_By_m').getData()/init_UB
UBz = S.Scalar('Uelm_Bz_m').getData()/init_UB

UEx = S.Scalar('Uelm_Ex').getData()/(c**2*init_UB)
UEy = S.Scalar('Uelm_Ey').getData()/(c**2*init_UB)
UEz = S.Scalar('Uelm_Ez').getData()/(c**2*init_UB)
field1_t = np.array(S.Scalar('Uelm_By_m').getTimes())


# particles energy
Ukin_shell_ion = S.Scalar('Ukin_shell_ion').getData()
Ukin_cold_ion = S.Scalar('Ukin_cold_ion').getData()


# 单独计算 \delta Bx
# UBx
UBx = np.zeros(len(field0_t))
delta_Bx = Bx - B0        

for i in range(len(field0_t)):
    UBx[i] = np.sum(0.5 * np.power(delta_Bx[i,:,:],2)*dx*dy)
UBx /= init_UB

In [ ]:
# 数据截断至450
# 转换时间单位
t_snapshot = 450
t0_normalized = field0_t * wcp
t1_normalized = field1_t * wcp

# 2. 找到时间等于 450 对应的索引位置
# 使用 np.searchsorted 可以高效找到最接近 450 的索引
idx0_s = np.searchsorted(t0_normalized, t_snapshot)
idx1_s = np.searchsorted(t1_normalized, t_snapshot)

# 1. Scalar

In [ ]:
# 对数组进行切片截断 [0:idx]
# 截断 Field 相关数据 (Bx, UBx, field0_t)
field0_t = field0_t[:idx0_s]
UBx = UBx[:idx0_s]
# 如果 Bx 很大，截断它能节省内存
Bx = Bx[:idx0_s, :, :] 

# 截断 Scalar 相关数据 (UBy, UBz, UEx, UEy, UEz, Ukin, field1_t)
field1_t = field1_t[:idx1_s]
UBy = UBy[:idx1_s]
UBz = UBz[:idx1_s]
UEx = UEx[:idx1_s]
UEy = UEy[:idx1_s]
UEz = UEz[:idx1_s]
Ukin_shell_ion = Ukin_shell_ion[:idx1_s]
Ukin_cold_ion = Ukin_cold_ion[:idx1_s]

In [10]:
with h5py.File('./data/' + filename + '_scalar.h5','a') as f:
    f.create_dataset('field0_t', data=field0_t)
    f.create_dataset('field1_t', data=field1_t)
    f.create_dataset('UBx', data=UBx)
    f.create_dataset('UBy', data=UBy)
    f.create_dataset('UBz', data=UBz)
    f.create_dataset('UEx', data=UEx)
    f.create_dataset('UEy', data=UEy)
    f.create_dataset('UEz', data=UEz)
    f.create_dataset('Ukin_shell_ion ', data=Ukin_shell_ion)
    f.create_dataset('Ukin_cold_ion ', data=Ukin_cold_ion)

del UBx,UBy,UBz,UEx,UEy,UEz,Utot,Ukin_cold_ion,Ukin_shell_ion

# 2. Particle Binning

In [11]:
T0 = mp*vbth**2

Pxx_c = np.array(S.ParticleBinning(0).getData())
Pyy_c = np.array(S.ParticleBinning(1).getData())
Pzz_c = np.array(S.ParticleBinning(2).getData())
ntot_c = np.array(S.ParticleBinning(3).getData())

Pxx_s = np.array(S.ParticleBinning(4).getData())
Pyy_s = np.array(S.ParticleBinning(5).getData())
Pzz_s = np.array(S.ParticleBinning(6).getData())
ntot_s = np.array(S.ParticleBinning(7).getData())

Pxx_e = np.array(S.ParticleBinning(8).getData())
Pyy_e = np.array(S.ParticleBinning(9).getData())
Pzz_e = np.array(S.ParticleBinning(10).getData())
ntot_e = np.array(S.ParticleBinning(11).getData())

binning_t = np.array(S.ParticleBinning(11).getTimes())

Tpa_c = Pxx_c/ntot_c / T0
Tpa_s = Pxx_s/ntot_s / T0
Tpa_e = Pxx_e/ntot_e / T0

Tpe_c = (Pyy_c+Pzz_c)/ntot_c/2 / T0 
Tpe_s = (Pyy_s+Pzz_s)/ntot_s/2 / T0 
Tpe_e = (Pyy_e+Pzz_e)/ntot_e/2 / T0 

A_c = Tpe_c/Tpa_c - 1
A_s = Tpe_s/Tpa_s - 1
A_e = Tpe_e/Tpa_e - 1

In [ ]:
# 数据截断至450
# 转换时间单位
t_snapshot = 450
t_binning_normalized = binning_t * wcp


# 找到时间等于 450 对应的索引位置
# every_particleBinning = 100
# 使用 np.searchsorted 可以高效找到最接近 450 的索引
idx_binning_s = np.searchsorted(t_binning_normalized, t_snapshot)

# 对数组进行切片截断 [0:idx]
# 截断 particle binning 相关数据 (binning_t, Tpa_c, Tpe_c, A_c)
binning_t = binning_t[:idx_binning_s]
Tpa_c = UBy[:idx_binning_s]
Tpe_c = UBz[:idx_binning_s]
A_c = UEx[:idx_binning_s]

In [13]:
with h5py.File('./data/' + filename + '_scalar.h5','a') as f:
    f.create_dataset('binning_t', data=binning_t)
    f.create_dataset('Tpa_c', data=Tpa_c)
    f.create_dataset('Tpe_c', data=Tpe_c)
    f.create_dataset('A_c', data=A_c)

del Tpa_c,Tpa_s,Tpa_e,Tpe_c,Tpe_s,Tpe_e,A_c,A_s,A_e
del Pxx_c,Pxx_s,Pxx_e
del Pyy_c,Pyy_s,Pyy_e
del Pzz_c,Pzz_s,Pzz_e
del ntot_c,ntot_s,ntot_e

# 3. wave properties

In [14]:
# Bx and time have been read in scalar data processing
# read field0
By = np.array(S.Field(0,'By').getData())
Bz = np.array(S.Field(0,'Bz').getData())

In [17]:
with h5py.File('./data/ns20.h5','a') as f:
    f.create_dataset('Bx', data=Bx)
    f.create_dataset('By', data=By)
    f.create_dataset('Bz', data=Bz)

del bx_space,by_space,bz_space
del xpsd_space,ypsd_space,zpsd_space
del bx,by,bz
del xpsd, ypsd, zpsd